In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Imports

In [0]:
import sys
sys.path.append('/content/drive/My Drive/Colab Notebooks')

In [4]:
import feature_string as feature_string
import feature_num as feature_num
import feature_categorica as feature_categorica
import common_machine_learning as common
import my_pipeline as my_pipe

import pandas as pd
import numpy as np

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.base import BaseEstimator, TransformerMixin

from hyperopt import fmin, tpe, hp, STATUS_OK

from xgboost import XGBRegressor

import time

import nltk
nltk.download('stopwords')
from nltk import pos_tag
from nltk.corpus import stopwords

import re

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [0]:
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)

## Constates

In [0]:
TARGET = 'precio'
K = 3
RANDOM_SEMILLA = 3 # numero favorito (de la suerte)
N_ITER = 33

## Train

In [0]:
train = common.cargar_set_optimizado('/content/drive/My Drive/Colab Notebooks/sets_de_datos/train.csv')
X_test = common.cargar_set_optimizado('/content/drive/My Drive/Colab Notebooks/sets_de_datos/train.csv')

## Features

In [0]:
TAG_RE = re.compile(r'<[^>]+>')
def remove_tags(text):
    return TAG_RE.sub('', text)

stop_words_sp = set(stopwords.words('spanish'))
stop_words_en = set(stopwords.words('english'))
stop_words = stop_words_sp | stop_words_en

In [0]:
train['descripcion'] = train['descripcion'].fillna('e')\
    .map(lambda x: remove_tags(str(x)))\
    .map(lambda x: "".join([letra.lower() for letra in x if letra.isalpha() or letra == ' ']))\
    .map(lambda x: " ".join([palabra for palabra in x.split() if palabra not in stop_words_sp and len(palabra)>2]))

In [0]:
X_test['descripcion'] = train['descripcion'].fillna('e')\
    .map(lambda x: remove_tags(str(x)))\
    .map(lambda x: "".join([letra.lower() for letra in x if letra.isalpha() or letra == ' ']))\
    .map(lambda x: " ".join([palabra for palabra in x.split() if palabra not in stop_words_sp and len(palabra)>2]))

In [0]:
train['precio_categorico'] = train['precio'] /10000

In [0]:
def categorizar_precio(precio):
    categorias = [100,200,300,400,500]
    for i in range(len(categorias)):
        if precio < categorias[i]:
            return i
    return 4

In [0]:
train['categoria'] = train['precio_categorico'].transform(lambda x: categorizar_precio(x))

In [0]:
train = train.drop(['precio_categorico'], axis = 1)

In [0]:
count_vec = CountVectorizer()
tf_idf_transformer = TfidfTransformer()
clasificador = MultinomialNB()

In [23]:
train_descripcion_counts = count_vec.fit_transform(train['descripcion'])
train_descripcion_tfidf = tf_idf_transformer.fit_transform(train_descripcion_counts)
clasificador.fit(train_descripcion_tfidf, train['categoria'])

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [0]:
X_test_descripcion_counts = count_vec.transform(X_test['descripcion'])
X_test_descripcion_tfidf = tf_idf_transformer.transform(X_test_descripcion_counts)
X_test['categoria'] = clasificador.predict(X_test_descripcion_tfidf)

In [0]:
train.set_index('id', inplace = True)
X_test.set_index('id', inplace = True)

In [0]:
train_categorias = train[['categoria']]
X_test_categorias = X_test[['categoria']]

In [0]:
train_categorias.to_csv('/content/drive/My Drive/Colab Notebooks/sets_de_datos/dima_train_categorias_descripciones.csv')

In [0]:
X_test_categorias.to_csv('/content/drive/My Drive/Colab Notebooks/sets_de_datos/dima_test_categorias_descripciones.csv')

In [32]:
pd.read_csv('/content/drive/My Drive/Colab Notebooks/sets_de_datos/dima_train_categorias_descripciones.csv')

,id,categoria
0,254099,2
1,53461,3
2,247984,1
3,209067,0
4,185997,1
...,...,...
239995,119879,0
239996,259178,1
239997,131932,3
239998,146867,2


In [33]:
pd.read_csv('/content/drive/My Drive/Colab Notebooks/sets_de_datos/dima_test_categorias_descripciones.csv')

,id,categoria
0,254099,1
1,53461,1
2,247984,1
3,209067,0
4,185997,0
...,...,...
239995,119879,0
239996,259178,1
239997,131932,1
239998,146867,1
